In [4]:
class BTree(object):
    # make node a nested class for easy sake
    class Node(object):
        def __init__(self, t):
            # make reference to degree t, keys in node, children of node, and if node is a leaf
            self._t = t
            self.keys = []
            self.children = []
            self.leaf = True

        def split(self, parent, new_val):
            # very important for insert and delete, basically an implementation of the pseudo code from class (roughly)
            
            # allocate node and determine split and key getting pushed up
            new_node = self.__class__(self._t)
            #__class__ refers to node in this case
            mid_point = len(self.keys) // 2
            split_value = self.keys[mid_point]
            parent.add_key(split_value)
            # new node inherits the original node's children and half of the original keys
            new_node.children = self.children[mid_point + 1:]
            self.children = self.children[:mid_point + 1]
            new_node.keys = self.keys[mid_point + 1:]
            self.keys = self.keys[:mid_point]

            # set leaf to false to false if children are present
            if len(new_node.children) > 0:
                new_node.leaf = False

            # the parent will then receive the new node
            parent.children = parent.add_child(new_node)
            
            
            if new_val < split_value:
                return self
            else:
                return new_node

        # for readablity sake
        def add_key(self, value):
            self.keys.append(value)
            self.keys.sort()

        # same with this one :/
        def add_child(self, new_node):
            i = len(self.children) - 1
            while i >= 0 and self.children[i].keys[0] > new_node.keys[0]:
                i -= 1
            return self.children[:i + 1] + [new_node] + self.children[i + 1:]

    # constructing the btree object, just need the degree creating the first node
    # min degree of btree is 2 so that will be default
    def __init__(self, t=2):
        self._t = t
        self.root = self.Node(t)

    def insert(self, new_val):
        node = self.root
        # first case will be if the node is full
        if len(node.keys) == 2 * node._t - 1:
            # create node that will take value that will be pushed
            new_root = self.Node(self._t)
            new_root.children.append(self.root)
            new_root.leaf = False
            node = node.split(new_root, new_val)
            self.root = new_root

        # if node isn't full and is not leaf
        while not node.leaf:
            i = len(node.keys) - 1
            # check what node the new value fits in, since there are t+1 different children maximum
            # we just check for the first key that is larger than the new value
            # then the new value will be placed in the appropriate child node
            while i >= 0 and new_val < node.keys[i]:
                i -= 1
            i += 1

            # check children, if it is full split the node to make room
            next_node = node.children[i]
            if len(next_node.keys) == 2 * node._t - 1:
                node = next_node.split(node, new_val)
            else:
                node = next_node
        # lastly add the new value
        node.add_key(new_val)

    def search(self, value, node=None):
        if node is None:
            node = self.root
        # if found then return array of keys
        if value in node.keys:
            return node.keys
        elif node.leaf:
        # if the node is a leaf that means there is nothing else to search
        # thus the key doesn't exist
            return "NULL"
        else:
        # if all is false meaning there is more to search then recusively call function
            i = 0
            while i < len(node.keys) and value > node.keys[i]:
                i += 1
            return self.search(value, node.children[i])

    
    def print_order(self):
        current = [self.root]
        # i think i could have done this recursively but a while seamed simpler
        # iterate until the array is empty
        while current:
            next = []
            output = ""
            for node in current:
                if node.children:
                    next.extend(node.children)
                # add keys to output string
                output += str(node.keys) + " "
            print(output)
            # set current to children array
            current = next

    # god why is delete so much more complicated
    def delete(self, value):
        self._delete(self.root, value)
        # end checkers if there are no more keys in the node
        # set smallest child array to orig
        if len(self.root.keys) == 0: 
            if len(self.root.children) > 0:
                self.root = self.root.children[0]
            else:
                self.root = None

    # different cases of deletion
    def _delete(self, node, value):
        t = self._t
        idx = self._find_key_index(node, value)

        if idx < len(node.keys) and node.keys[idx] == value: # check if value is in node
            # if the node is a leaf then just delete
            # BEST case scenario
            if node.leaf:
                node.keys.pop(idx)
            else:
                # or the node is not a lead (internal node
                # case 1 if the left child node has atleast t keys then replace the deleted value with
                # the highest value in the left child
                if len(node.children[idx].keys) >= t:
                    pred = self._get_predecessor(node, idx)
                    node.keys[idx] = pred
                    self._delete(node.children[idx], pred)
                # case 2 if the left child does not have atleast t keys, we take from the right child
                # and replace the value like we did in case 1
                elif len(node.children[idx + 1].keys) >= t:
                    succ = self._get_successor(node, idx)
                    node.keys[idx] = succ
                    self._delete(node.children[idx + 1], succ)
                # case 3 both predecesor and succesor do not have enough keys to replace the deleted
                # value, we then merge then together and the delete
                else:
                    self._merge(node, idx)
                    self._delete(node.children[idx], value)
        
        else:
            if node.leaf:
                return

            flag = idx == len(node.keys)
            if len(node.children[idx].keys) < t:
                if idx != 0 and len(node.children[idx - 1].keys) >= t:
                    self._borrow_from_prev(node, idx)
                elif idx != len(node.children) - 1 and len(node.children[idx + 1].keys) >= t:
                    self._borrow_from_next(node, idx)
                else:
                    if idx != len(node.children) - 1:
                        self._merge(node, idx)
                    else:
                        self._merge(node, idx - 1)

            # recursively call function until it finds the node that contains the value
            if flag and idx > len(node.keys):
                self._delete(node.children[idx - 1], value)
            else:
                self._delete(node.children[idx], value)

    def _find_key_index(self, node, value): # find the placement of value in node
        idx = 0
        while idx < len(node.keys) and node.keys[idx] < value:
            idx += 1
        return idx
    # idx will return out of bounds if value is not found

    def _get_predecessor(self, node, idx): # return the highest value in the left child
        cur = node.children[idx]
        while not cur.leaf:
            cur = cur.children[-1]
        return cur.keys[-1]

    def _get_successor(self, node, idx): # get lowest value in the right child
        cur = node.children[idx + 1]
        while not cur.leaf:
            cur = cur.children[0]
        return cur.keys[0]

    def _merge(self, node, idx): # combine pred and succ 
        child = node.children[idx]
        sibling = node.children[idx + 1]

        child.keys.append(node.keys[idx])
        child.keys.extend(sibling.keys)

        if len(sibling.children) > 0:
            child.children.extend(sibling.children)

        node.keys.pop(idx)
        node.children.pop(idx + 1)

    def _borrow_from_prev(self, node, idx):
        child = node.children[idx]
        sibling = node.children[idx - 1]

        child.keys.insert(0, node.keys[idx - 1])
        if not child.leaf:
            child.children.insert(0, sibling.children.pop())
        node.keys[idx - 1] = sibling.keys.pop()

    def _borrow_from_next(self, node, idx):
        child = node.children[idx]
        sibling = node.children[idx + 1]

        child.keys.append(node.keys[idx])
        if not child.leaf:
            child.children.append(sibling.children.pop(0))
        node.keys[idx] = sibling.keys.pop(0)

In [5]:
import unittest
# thank god there was a unittest file on google colab :D

class Test_code(unittest.TestCase):

    def create(self):
        self.btree = BTree(t=2)

    def test_insert(self):
        #insert seamingly random numbers to tree
        values = [10, 20, 5, 6, 12, 30, 7, 17]
        for value in values:
            self.btree.insert(value)
        
        # test if structure of true is equal to answer
        self.assertEqual(self.btree.root.keys, [10, 20])
        self.assertEqual(self.btree.root.children[0].keys, [5, 6, 7])
        self.assertEqual(self.btree.root.children[1].keys, [12, 17])
        self.assertEqual(self.btree.root.children[2].keys, [30])

    def test_search(self):
        values = [10, 20, 5, 6, 12, 30, 7, 17]
        for value in values:
            self.btree.insert(value)

        # same test case but now its searching if the node containing the desired value is correct
        self.assertEqual(self.btree.search(6), [5, 6, 7])
        self.assertEqual(self.btree.search(20), [10, 20])
        self.assertEqual(self.btree.search(30), [30])
        self.assertEqual(self.btree.search(100), "NULL")

    def test_delete_leaf(self):
        values = [10, 20, 5, 6, 12, 30, 7, 17]
        for value in values:
            self.btree.insert(value)

        # test if delete actually deletes the value from the node
        self.btree.delete(6)
        self.assertEqual(self.btree.search(6), "NULL")
        self.assertEqual(self.btree.root.children[0].keys, [5, 7])

    def test_delete_internal(self):
        values = [10, 20, 5, 6, 12, 30, 7, 17]
        for value in values:
            self.btree.insert(value)

        # test if deleting a root key doesn't brick the tree
        self.btree.delete(10)
        self.assertEqual(self.btree.search(10), "NULL")
        self.assertEqual(self.btree.root.keys, [7, 20])

    def test_delete_root(self):
        values = [10, 20, 5, 6, 12, 30, 7, 17]
        for value in values:
            self.btree.insert(value)

        # delete everything for fun
        for value in values:
            self.btree.delete(value)

        self.assertIsNone(self.btree.root)  # Root should be None after deleting all elements

In [6]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

EEEEE
ERROR: test_delete_internal (__main__.Test_code.test_delete_internal)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\dzhan\AppData\Local\Temp\ipykernel_11048\3688636100.py", line 45, in test_delete_internal
    self.btree.insert(value)
    ^^^^^^^^^^
AttributeError: 'Test_code' object has no attribute 'btree'

ERROR: test_delete_leaf (__main__.Test_code.test_delete_leaf)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\dzhan\AppData\Local\Temp\ipykernel_11048\3688636100.py", line 35, in test_delete_leaf
    self.btree.insert(value)
    ^^^^^^^^^^
AttributeError: 'Test_code' object has no attribute 'btree'

ERROR: test_delete_root (__main__.Test_code.test_delete_root)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\dzhan\AppData\Local\Temp\ipykernel_11048\3688